### Importing necessary libraries

In [83]:
from dotenv import load_dotenv
import os
from langchain_core.prompts.prompt import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
import json
from pathlib import Path
from operator import itemgetter

### Loading Google API Key from environment variables

In [34]:
# Load environment variables from .env file
load_dotenv()

# Set the GOOGLE_API_KEY environment variable to the value retrieved from .env file
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

### Initializing Large Language Model

In [22]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

### Loading JSON response file

In [42]:
path_absolute = Path("E:\Learnings\Comprehension-generator")
path_relative = Path("response.json")
full_path = path_absolute / path_relative

with open(full_path,'r') as f:
   RESPONSE_JSON = json.load(f)

In [43]:
RESPONSE_JSON

{'comprehension_passage': 'This is a sample comprehension passage. It contains several sentences providing information or a story.',
 'mcqs': [{'question': 'What is the capital of France?',
   'options': ['London', 'Paris', 'Berlin', 'Rome'],
   'answer_key': ''},
  {'question': "Who wrote 'To Kill a Mockingbird'?",
   'options': ['Harper Lee',
    'J.K. Rowling',
    'Stephen King',
    'Charles Dickens'],
   'answer_key': 0}],
 'short_answer_questions': [{'question': 'What is the main theme of the passage?',
   'demo_answer': 'The main theme of the passage is about...'},
  {'question': 'Explain the significance of the character X in the passage.',
   'demo_answer': 'The character X is significant because...'}],
 'long_answer_questions': [{'question': 'Discuss the impact of climate change on global agriculture.',
   'demo_answer': 'Climate change has a profound impact on global agriculture...'},
  {'question': 'Analyze the character development in the provided passage.',
   'demo_answ

In [69]:
generate_instruction= """
You are an expert comprehension generator\ Given a specific {topic} your task is to\  
generate a comprehension passage within {words} words \
keeping in mind the difficulty level {difficulty}
Next you have to have to generate MCQS, short answer type and long answer type questions.
The number of questions for MCQ will be {mcq_no},\
for short answer type{sa_no} and\
for long answer type{la_no}.
Provide the demo answers for all the generated questions of MCQ, short answer and long answer questions at the end.
The long type question demo answers should be of minimimum 100 words\
Provide the output in the given format\
{RESPONSE_JSON}
The #RESPONSE_JSON keys and values should be enclosed within # double quotes""
"""

In [70]:
generation_prompt= PromptTemplate(
    input_variables=['topic','words','difficulty','mcq_no','sa_no','la_no','RESPONSE_JSON'],
    template= generate_instruction
)

In [71]:
generation_prompt

PromptTemplate(input_variables=['RESPONSE_JSON', 'difficulty', 'la_no', 'mcq_no', 'sa_no', 'topic', 'words'], template='\nYou are an expert comprehension generator\\ Given a specific {topic} your task is to\\  \ngenerate a comprehension passage within {words} words keeping in mind the difficulty level {difficulty}\nNext you have to have to generate MCQS, short answer type and long answer type questions.\nThe number of questions for MCQ will be {mcq_no},for short answer type{sa_no} andfor long answer type{la_no}.\nProvide the demo answers for all the generated questions of MCQ, short answer and long answer questions at the end.\nThe long type question demo answers should be of minimimum 100 wordsProvide the output in the given format{RESPONSE_JSON}\nThe #RESPONSE_JSON keys and values should be enclosed within # double quotes""\n')

In [72]:
output_parser = StrOutputParser()

In [73]:
chain_generation=generation_prompt| llm| output_parser 

In [74]:
chain_generation

PromptTemplate(input_variables=['RESPONSE_JSON', 'difficulty', 'la_no', 'mcq_no', 'sa_no', 'topic', 'words'], template='\nYou are an expert comprehension generator\\ Given a specific {topic} your task is to\\  \ngenerate a comprehension passage within {words} words keeping in mind the difficulty level {difficulty}\nNext you have to have to generate MCQS, short answer type and long answer type questions.\nThe number of questions for MCQ will be {mcq_no},for short answer type{sa_no} andfor long answer type{la_no}.\nProvide the demo answers for all the generated questions of MCQ, short answer and long answer questions at the end.\nThe long type question demo answers should be of minimimum 100 wordsProvide the output in the given format{RESPONSE_JSON}\nThe #RESPONSE_JSON keys and values should be enclosed within # double quotes""\n')
| ChatGoogleGenerativeAI(model='gemini-pro', client=genai.GenerativeModel(
      model_name='models/gemini-pro',
      generation_config={},
      safety_sett

In [140]:
review_instruction= """
As an expert in English grammar and writing, your task is to assess a comprehension passage{response}\
The passage provided is {words} words long and pertains to the topic of {topic}\
Your evaluation should include an assessment of the passage's difficulty level, as specified {difficulty}\
Furthermore, you are required to evaluate multiple-choice questions, short-answer questions, and long-answer questions embedded within the passage\
If any of the questions do not align with the cognitive and analytical abilities expected of the students, you must suggest changes to ensure they are suitable\
Additionally, you are to evaluate the provided answers to the questions, ensuring they are accurate and appropriate.
Please ensure that the output consists solely of the updated comprehension passage and the questions along with their answers\
as per the given format {RESPONSE_JSON}\
In the output format keys and values of  JSON  within # double quotes
"""

In [141]:
review_template= PromptTemplate(
    input_variables=['topic','words','difficulty','response','RESPONSE_JSON'],
    template= review_instruction
)

In [142]:
review_template

PromptTemplate(input_variables=['RESPONSE_JSON', 'difficulty', 'response', 'topic', 'words'], template="\nAs an expert in English grammar and writing, your task is to assess a comprehension passage{response}The passage provided is {words} words long and pertains to the topic of {topic}Your evaluation should include an assessment of the passage's difficulty level, as specified {difficulty}Furthermore, you are required to evaluate multiple-choice questions, short-answer questions, and long-answer questions embedded within the passageIf any of the questions do not align with the cognitive and analytical abilities expected of the students, you must suggest changes to ensure they are suitableAdditionally, you are to evaluate the provided answers to the questions, ensuring they are accurate and appropriate.\nPlease ensure that the output consists solely of the updated comprehension passage and the questions along with their answersas per the given format {RESPONSE_JSON}In the output format k

In [143]:
review_chain= (
    {"response": chain_generation, "difficulty": itemgetter("difficulty"), "topic": itemgetter("topic"), "words": itemgetter("words"), "RESPONSE_JSON": itemgetter("RESPONSE_JSON")}
    | review_template
    | llm
    | StrOutputParser() 
)

In [144]:
review_chain

{
  response: PromptTemplate(input_variables=['RESPONSE_JSON', 'difficulty', 'la_no', 'mcq_no', 'sa_no', 'topic', 'words'], template='\nYou are an expert comprehension generator\\ Given a specific {topic} your task is to\\  \ngenerate a comprehension passage within {words} words keeping in mind the difficulty level {difficulty}\nNext you have to have to generate MCQS, short answer type and long answer type questions.\nThe number of questions for MCQ will be {mcq_no},for short answer type{sa_no} andfor long answer type{la_no}.\nProvide the demo answers for all the generated questions of MCQ, short answer and long answer questions at the end.\nThe long type question demo answers should be of minimimum 100 wordsProvide the output in the given format{RESPONSE_JSON}\nThe #RESPONSE_JSON keys and values should be enclosed within # double quotes""\n')
            | ChatGoogleGenerativeAI(model='gemini-pro', client=genai.GenerativeModel(
                  model_name='models/gemini-pro',
       

In [145]:
response= review_chain.invoke({'RESPONSE_JSON':RESPONSE_JSON, 'difficulty':'easy', 'la_no':2, 'mcq_no':3, 'sa_no':1, 'topic':'India', 'words':300 })

In [148]:
print(response)

{
 "comprehension_passage": "India is a vast and diverse country located in South Asia. It is the seventh-largest country in the world by landmass and the second-most populous country with over 1.3 billion people. India is a federal republic consisting of 28 states and 8 union territories. The capital of India is New Delhi.\n\nIndia has a rich history and culture. The Indus Valley Civilization, one of the world's oldest civilizations, flourished in the Indian subcontinent from 2600 to 1900 BCE. India was ruled by various empires and dynasties over the centuries, including the Mauryan Empire, the Gupta Empire, and the Mughal Empire. India gained independence from British rule in 1947.",
 "mcqs": [
  {
   "question": "What is the capital of India?",
   "options": [
    "Mumbai",
    "New Delhi",
    "Kolkata",
    "Chennai"
   ],
   "answer_key": "New Delhi"
  },
  {
   "question": "Which of the following is not a state in India?",
   "options": [
    "Uttar Pradesh",
    "Karnataka",
  

In [149]:
response= json.loads(response)

In [150]:
response

{'comprehension_passage': "India is a vast and diverse country located in South Asia. It is the seventh-largest country in the world by landmass and the second-most populous country with over 1.3 billion people. India is a federal republic consisting of 28 states and 8 union territories. The capital of India is New Delhi.\n\nIndia has a rich history and culture. The Indus Valley Civilization, one of the world's oldest civilizations, flourished in the Indian subcontinent from 2600 to 1900 BCE. India was ruled by various empires and dynasties over the centuries, including the Mauryan Empire, the Gupta Empire, and the Mughal Empire. India gained independence from British rule in 1947.",
 'mcqs': [{'question': 'What is the capital of India?',
   'options': ['Mumbai', 'New Delhi', 'Kolkata', 'Chennai'],
   'answer_key': 'New Delhi'},
  {'question': 'Which of the following is not a state in India?',
   'options': ['Uttar Pradesh', 'Karnataka', 'Nepal', 'Rajasthan'],
   'answer_key': 'Nepal'}